In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import dlc_bci as bci
from dlc_practical_prologue import *

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#from torch import nn
from torch.autograd import Variable


Load dataset

In [ ]:
# Load dataset
tr_input, tr_target = bci.load("bci", train=True, one_khz=False)
te_input, te_target = bci.load("bci", train=False, one_khz=False)

# rearrange data [batch size, timesteps, channels]
tr_input = np.transpose(tr_input, (0,2,1))
te_input = np.transpose(te_input, (0,2,1))

# Prepare onehot vector for computation of loss
tr_target_onehot = convert_to_one_hot_labels(tr_input, tr_target)
te_target_onehot = convert_to_one_hot_labels(te_input, te_target)

Standardize

In [ ]:
# normalize
tr_input = torch.nn.functional.normalize(tr_input, p=2, dim=0) 
te_input = torch.nn.functional.normalize(te_input, p=2, dim=0) 

# Convert to pytorch variable
tr_input, tr_target, tr_target_onehot = autograd.Variable(tr_input), Variable(tr_target), Variable(tr_target_onehot)
te_input, te_target, te_target_onehot = autograd.Variable(te_input), Variable(te_target), Variable(te_target_onehot)

Output of NN is still a vector (cross entropy loss handles log-softmax)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        batch_size = 4
        feature_dim = 28
        hidden_size = 20
        num_layers = 1
        
        self.lstm = nn.LSTM(feature_dim, hidden_size=hidden_size, batch_first=True)
        self.h0 = Variable(torch.randn(1, batch_size, hidden_size)) 
        self.c0 = Variable(torch.randn(1, batch_size, hidden_size))
                
    def forward(self, x, mode=False):
                
        output, hn = self.lstm(x, (self.h0,self.c0))  
        
        print(np.shape(output))    
        return output

In [ ]:
# construct and train model
model = Net()
tr_loss, te_loss = bci.train_model(model, tr_input, tr_target, 4, te_input, te_target, 4, 200)

# compute train and test errors
nb_tr_errors = bci.compute_nb_errors(model, tr_input, tr_target_onehot, 4)
nb_te_errors = bci.compute_nb_errors(model, te_input, te_target_onehot, 4)

print('tr error {:0.2f}% {:d}/{:d}'.format((100 * nb_tr_errors) / tr_input.size(0), nb_tr_errors, tr_input.size(0)))
print('te error {:0.2f}% {:d}/{:d}'.format((100 * nb_te_errors) / te_input.size(0), nb_te_errors, te_input.size(0)))